In [1]:
import torch
import torch.nn as nn

In [4]:
x = torch.rand(64,1,400,128)
y = x[:,:,::4,:]
y.shape

torch.Size([64, 1, 100, 128])

In [5]:
z = torch.rand(64,1,100,128)

torch.cat((x,z),dim=-2).shape

torch.Size([64, 1, 500, 128])

In [34]:
x = torch.rand(64,2048,25,4)
y = torch.rand(64, 256, 100, 4)
m1 = nn.AvgPool2d(x.size()[2:])
m2 = nn.AvgPool2d(y.size()[2:])
m = nn.AdaptiveAvgPool2d((1, 1))
out1 = m(x)
out2 = m(y)

In [35]:
pool_out = [out1, out2]
out = torch.cat(pool_out, 1)

In [36]:
out = out.permute((0, 2, 3, 1))
out.shape

torch.Size([64, 1, 1, 2304])

In [27]:
out.mean([1, 2]).shape

torch.Size([64, 2304])

In [30]:
out.view(x.shape[0], -1).shape

torch.Size([64, 2304])

In [38]:
[8,16,32] // 4

TypeError: unsupported operand type(s) for //: 'list' and 'int'

In [53]:
conv1 = nn.Conv2d(1,8,[5,7], stride=[2,2], padding=[2,3])
x = torch.rand(12,1,400,128)
x = conv1(x)
x.shape

torch.Size([12, 8, 200, 64])

In [56]:
pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
pool1(x).shape

torch.Size([12, 8, 100, 32])

In [67]:
class FuseFastToSlow(nn.Module):
    """
    Fuses the information from the Fast pathway to the Slow pathway. Given the
    tensors from Slow pathway and Fast pathway, fuse information from Fast to
    Slow, then return the fused tensors from Slow and Fast pathway in order.
    """

    def __init__(self, dim_in, alpha, fusion_conv_channel_ratio, fusion_kernel=7):

        super(FuseFastToSlow, self).__init__()
        self.conv_f2s = nn.Conv2d(
            dim_in,
            dim_in * fusion_conv_channel_ratio,
            kernel_size=[fusion_kernel, 1],
            stride=[alpha, 1],
            padding=[fusion_kernel // 2, 0]
        )
        self.bn = nn.BatchNorm2d(dim_in * fusion_conv_channel_ratio)
        self.relu = nn.ReLU()

    def forward(self, x_s, x_f):
        fuse = self.conv_f2s(x_f)
        fuse = self.bn(fuse)
        fuse = self.relu(fuse)
        x_s_fuse = torch.cat([x_s, fuse], 1)
        return x_s_fuse, x_f

In [68]:
lateral_conv1 = FuseFastToSlow(dim_in=8, alpha=4, fusion_conv_channel_ratio=2)
x_s = torch.rand(12, 64, 25, 32)
x_f = torch.rand(12, 8, 100, 32)

a , b = lateral_conv1(x_s, x_f)

print(a.shape)
print(b.shape)

torch.Size([12, 80, 25, 32])
torch.Size([12, 8, 100, 32])


In [69]:
16*64 // 8

128